Victor CHAU & Tahar AMAIRI
# IFT 870 - Forage de données
## TP3 : onctions prédictives

### Sommaire

[TODO]

## Importation des données

In [3]:
# import libs
from matplotlib import pyplot as plt
from IPython.display import display
import seaborn as sns
import pandas as pd
import numpy as np

# read CSV
data = pd.read_csv('TP3_data.csv', index_col=0)

# cast classe column to in
data['Classe'] = data['Classe'].astype(int)

# print the df
data.head()

,Attribut1,Attribut2,Attribut3,Attribut4,Classe
0,-8.850074,4.041708,-4.545313,-4.578589,18
1,1.009469,-9.347073,-6.515912,9.669842,5
2,5.939861,0.718491,-0.564980,3.312923,8
3,5.629454,1.395495,-0.424261,4.603658,8
4,-6.229520,3.496113,2.202815,10.431126,10


## 1. Implémentation de fonctions
### a) model_score(X,y,class_model,params) 